In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [2]:
#  Esta vez mantengamos las convenciones para guardar los csvs, y los nombres de los archivos.
#  El dataset nuevo que nos dio Trocafone lo renombré a events.csv por comodidad.

df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/features_vistos.csv', low_memory=False, index_col='person')
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2)
df_unidos = df_unidos.join(df3)

#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')

In [3]:
df_test.head()

,vio_productos,tiene_checkouts,tiene_conversions,cant_prod_vistos,cantidad_checkouts,cantidad_conversions,compro_Apple,compro_Asus,compro_LG,compro_Lenovo,...,compro_Sony,vio_mas_Apple,vio_mas_Asus,vio_mas_LG,vio_mas_Lenovo,vio_mas_Motorola,vio_mas_Quantum,vio_mas_Samsung,vio_mas_Sony,label
person,,,,,,,,,,,,,,,,,,,,,
ad93850f,True,True,False,20.0,1.0,0.0,False,False,False,False,...,False,True,False,False,False,False,False,False,False,0
1b9f7cf6,True,True,False,9.0,1.0,0.0,False,False,False,False,...,False,True,False,False,False,False,False,False,False,0
de8fe91b,True,True,False,27.0,1.0,0.0,False,False,False,False,...,False,False,False,False,False,False,False,True,False,0
45baf068,True,True,False,6.0,1.0,0.0,False,False,False,False,...,False,False,False,False,False,False,False,True,False,0
99abca5a,True,True,True,630.0,18.0,2.0,False,False,False,False,...,False,False,False,False,False,True,False,False,False,0


In [4]:
features = df_test.columns.tolist()
features.remove('label')
print(features)

['vio_productos', 'tiene_checkouts', 'tiene_conversions', 'cant_prod_vistos', 'cantidad_checkouts', 'cantidad_conversions', 'compro_Apple', 'compro_Asus', 'compro_LG', 'compro_Lenovo', 'compro_Motorola', 'compro_Quantum', 'compro_Samsung', 'compro_Sony', 'vio_mas_Apple', 'vio_mas_Asus', 'vio_mas_LG', 'vio_mas_Lenovo', 'vio_mas_Motorola', 'vio_mas_Quantum', 'vio_mas_Samsung', 'vio_mas_Sony']


## Veamos si las labels están balanceadas

In [5]:
labels['label'].value_counts(normalize=True)

0    0.949521
1    0.050479
Name: label, dtype: float64

# Entrenando el Random Forest

In [6]:
#  class_weight='balanced' hace que se le de la misma importancia a las clases a predecir, sin importar (creo)
#  su distribución.
#  Los valores de los parámetros los seleccioné probando, aún no sé bien cómo influencian los resultados.
rf = RandomForestClassifier(n_estimators=30, n_jobs=2, min_samples_split=200,\
                             random_state=0, class_weight='balanced')

#  Partimos los datos que tenemos para entrenar en dos partes, una para entrenar el modelo (80% de los datos)
#  y la otra parte se usará para probar el modelo (el 20% restante)
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20)

In [7]:
#  Fit entrena el modelo, y score corre una predicción y calcula un puntaje.

rf.fit(X_train,Y_train)
rf.score(X_test,Y_test)

0.7695081122843163

In [8]:
#  No sé bien cómo se evalúa este puntaje aún.

Y_pred = rf.predict(X_test)
print(roc_auc_score(Y_test, Y_pred))

0.758095064323638


In [9]:
#  Esto nos tira data sobre la precisión, cuántas de las cosas que clasificamos salieron bien clasificadas, y f1
#  no entendí bien qué es.

Y_truths = Y_test
print(classification_report(Y_truths, Y_pred))

             precision    recall  f1-score   support

          0       0.98      0.77      0.86      3671
          1       0.16      0.75      0.26       212

avg / total       0.94      0.77      0.83      3883



In [10]:
#  Esta matriz es la posta, miren esta respuesta de stack overflow para entender cómo funciona
#  https://stackoverflow.com/questions/30746460/how-to-interpret-scikits-learn-confusion-matrix-and-classification-report

print(confusion_matrix(Y_truths, Y_pred))

[[2830  841]
 [  54  158]]


In [11]:
feature_importances = pd.DataFrame(rf.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

In [12]:
#  Acá quedan ordenadas las features por importancia para el modelo, aparentemente la marca más vista y la marca
#  que más compró cada usuario no es tan importante.
feature_importances.head(20)

,importance
cantidad_checkouts,0.364544
cant_prod_vistos,0.257195
tiene_checkouts,0.251696
tiene_conversions,0.030860
cantidad_conversions,0.029923
compro_Samsung,0.016423
vio_mas_Apple,0.010123
vio_mas_Samsung,0.008479
compro_Apple,0.005614
vio_mas_Motorola,0.005469


# Preparamos el submit

In [13]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')

In [14]:
df_events.head()

,vio_productos,tiene_checkouts,tiene_conversions,cant_prod_vistos,cantidad_checkouts,cantidad_conversions,compro_Apple,compro_Asus,compro_LG,compro_Lenovo,...,compro_Samsung,compro_Sony,vio_mas_Apple,vio_mas_Asus,vio_mas_LG,vio_mas_Lenovo,vio_mas_Motorola,vio_mas_Quantum,vio_mas_Samsung,vio_mas_Sony
person,,,,,,,,,,,,,,,,,,,,,
4886f805,True,True,False,4.0,1.0,0.0,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
0297fc1e,True,True,False,404.0,7.0,0.0,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2d681dd8,True,True,False,13.0,1.0,0.0,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
cccea85e,True,True,False,739.0,1.0,0.0,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4c8a8b93,True,True,False,177.0,2.0,0.0,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [15]:
kaggle_pred = rf.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [16]:
#df_submit.to_csv('submit.csv')